In [55]:
!pip install qiskit --quiet
!pip install qiskit_nature --quiet
!pip install qiskit_ibm_runtime --quiet
!pip install --prefer-binary pyscf --quiet
import numpy as np
import pickle as pkl
import os
import itertools

# 02: H-Atom Fast Forward

Recent literature has shown methods such as Variational Fast Forwarding which aims to predict the long-term time evolutions of a system given data on the short-term evolutions of the system. Filip et al [1] and Cîrstoiu et al [2] have demonstrated two different ways to train a VFF using short-term evolution data. This dataset provides us short-term data and gives us testing points for long-term performance benchmarking

## 02.1 Data Format

The data generator gives us the following:
- `⎢ѱHK〉`: The Hartree Fock State of the system
- `X_train`: `[Δt, 2Δtm …MΔt]` which are uniform timestamps at which short-term evolutions are recorded
- `Y_train`: `U(Δt)⎢ѱHK〉...U(MΔt)⎢ѱHK〉` which are the short-term evolutions but with noise
- `X_test`: `[PΔt…NΔt]` which are the long-term timestamps
- `Y_test`: `U(PΔt)⎢ѱHK〉…U(NΔt)⎢ѱHK〉` which are the long-term noiseless evolutions

The data generator gets the `U(Δt)` and the `⎢ѱHK〉` from a H-atom model given by the hyperparameters. The noisy generations allow us to also benchmark our pipeline on how well it avoids overfitting to noise which wasn't discussed in the references above

### 02.1.1 H-Atom Hamiltonians from PySCF

We need some Chemistry to first decide what geometries to import.

- H2: Usually found at a `0.735 Å` equilibrium bond distance [O’Malley et al. (2016)]
- H3+: Usually found both in an equilateral triangle configuration with `0.9 Å` between each pair
- H6: Usually modelled as a linear chain of 6 atoms with bond lengths `1 Å` between each pair


In [56]:
H2 = {"atom":"H 0 0 0; H 0 0 0.735", "basis":"sto-3g", "charge":0, "spin":0}
H3 = {"atom":"H 0 0 -0.45; H 0 0 0.45; H 0 0.78 0;", "basis":"sto-3g", "charge":1, "spin":0}
H6 = {"atom":"H 0 0 0; H 0 0 1; H 0 0 2; H 0 0 3; H 0 0 4; H 0 0 5;", "basis":"sto-3g", "charge":0, "spin":0}

In [57]:
from pyscf import gto, scf
mol = gto.M(**H2)
mf = scf.RHF(mol).run(conv_tol=1e-12)
mol

converged SCF energy = -1.116998996754


Fermionic Hamiltonians are of the form:

$\hat{H}\;=\;E_{\text{nuc}}\;+\;\sum_{p q} h_{p q}\,a^{\dagger}_{p}\, a_{q}+\;\frac{1}{2}\,\sum_{p q r s}g_{p q r s}\,a^{\dagger}_{p}\, a^{\dagger}_{q}\,a_{r}\, a_{s}$

The first term is the constant nuclear repulsions term. The other terms are first quantization and second quantization terms for Fermionic Pair Creations and Anhilations. The co-efficients are derived from Molecular Orbital Integrals

#### Nuclear Repulsions Term

In [58]:
E_nuc = mol.energy_nuc()
E_nuc

np.float64(0.7199689944489797)

#### 1e and 2e Interactions in AO Basis

In [59]:
h_ao  = mf.get_hcore()
h_ao, h_ao.shape

(array([[-1.12421758, -0.9652574 ],
        [-0.9652574 , -1.12421758]]),
 (2, 2))

In [60]:
g_ao  = mol.intor('int2e',aosym='s8')
g_ao, g_ao.shape

(array([0.77460594, 0.44744572, 0.3009177 , 0.57187698, 0.44744572,
        0.77460594]),
 (6,))

#### Converting to MO Basis

If this works right, the results should be:


```
[[-1.25633907e+00, -1.37083854e-17],
[-6.07732712e-17, -4.71896007e-01]]

[[[[6.75710155e-01, 1.56722377e-16],
   [1.03976181e-16, 6.64581730e-01]],

  [[1.55375654e-16, 1.80931200e-01],
   [1.80931200e-01, 4.63164127e-16]]],


 [[[5.98074319e-17, 1.80931200e-01],
   [1.80931200e-01, 2.60271871e-16]],

  [[6.64581730e-01, 2.69194871e-16],
   [3.78920172e-16, 6.98573723e-01]]]]
```



In [61]:
C      = mf.mo_coeff
h_mo   = C.T @ h_ao @ C
h_mo

array([[-1.25633907e+00, -1.37083854e-17],
       [-6.07732712e-17, -4.71896007e-01]])

In [62]:
from pyscf import ao2mo
g_mo8 = ao2mo.kernel(mol, C)
g_mo  = ao2mo.restore(1, g_mo8, C.shape[1])
g_mo

array([[[[ 6.75710155e-01,  1.09338783e-16],
         [ 1.09338783e-16,  6.64581730e-01]],

        [[ 1.39486891e-16,  1.80931200e-01],
         [ 1.80931200e-01, -1.03094037e-16]]],


       [[[ 1.39486891e-16,  1.80931200e-01],
         [ 1.80931200e-01, -1.03094037e-16]],

        [[ 6.64581730e-01,  2.57172666e-16],
         [ 2.57172666e-16,  6.98573723e-01]]]])

#### Converting to SO Basis

In [63]:
n   = h_mo.shape[0]
n_so   = 2 * n
eps = 1e-10

h_so = np.zeros((n_so,n_so))
g_so = np.zeros((n_so,n_so,n_so,n_so))

# Mapping one-electron integrals
for p in range(n):
    for q in range(n):
        val = h_mo[p, q]
        if abs(val) < eps:
            continue
        # α spin
        h_so[2 * p, 2 * q] = val
        # β spin
        h_so[2 * p + 1, 2 * q + 1] = val

# Mapping two-electron integrals
for p in range(n):
    for q in range(n):
        for r in range(n):
            for s in range(n):
                val = g_mo[p, q, r, s]
                if abs(val) < eps:
                    continue
                # αααα
                g_so[2 * p, 2 * q, 2 * r, 2 * s] = val
                # ββββ
                g_so[2 * p + 1, 2 * q + 1, 2 * r + 1, 2 * s + 1] = val
                # αβαβ
                g_so[2 * p, 2 * q + 1, 2 * r, 2 * s + 1] = val
                # βαβα
                g_so[2 * p + 1, 2 * q, 2 * r + 1, 2 * s] = val

(E_nuc, h_so.shape, g_so.shape)

(np.float64(0.7199689944489797), (4, 4), (4, 4, 4, 4))

In [64]:
os.makedirs("_Hamiltonians", exist_ok=True)
with open("_Hamiltonians/H2_fermionic.bin", "wb") as f:
    pkl.dump((E_nuc, h_so, g_so), f)

### 02.1.2 Converting Fermionic Hamiltonian to Qubit Hamiltonian

Jordan-Wigner mapping is when we map the state of occupancy of the orbitals to one qubit each. So the below equation shows the tranform that we will apply:

$a_p ^\dagger = \frac 12 \left(Z_0 \otimes Z_1 \ ... Z_{p-1} \right) \otimes \left(X_p - j Y_p \right)$

The resulting Qubit Hamiltonian for H2 atom will be of the form:

$\hat H = c_0 I + c_1 Z_0 \ ... + \ c_5 Z_0 Z_1 + c_6 Z_0 Z_2 \ ... + \ c_{11} X_0X_1Y_2Y_3 + c_{12} X_0Y_1Y_2X_3 + c_{13} Y_0X_1X_2Y_3 + c_{13} Y_0Y_1X_2X_3$

Now each term can be taken to the Unitary with CNOTs, Rys and Rzs that depend on the coefficients.

We can define multiplication and addition for such symbolic terms to proceed

In [65]:
# (op1, op2) -> (phase, op_res)
mul_table = {
    ('I','I'):(1,'I'), ('I','X'):(1,'X'), ('I','Y'):(1,'Y'), ('I','Z'):(1,'Z'),
    ('X','I'):(1,'X'), ('X','X'):(1,'I'), ('X','Y'):(1j,'Z'),('X','Z'):(-1j,'Y'),
    ('Y','I'):(1,'Y'), ('Y','X'):(-1j,'Z'),('Y','Y'):(1,'I'),('Y','Z'):(1j,'X'),
    ('Z','I'):(1,'Z'), ('Z','X'):(1j,'Y'), ('Z','Y'):(-1j,'X'),('Z','Z'):(1,'I')
}

# pauli string is the list of single qubit gates for each qubit [Z,Z,Z,Z,X,I,I]
def a_p(p,n):
    # Anything can be represented as a sum of {string:coeff}
    str1 = tuple(['Z']*p + ['X'] + ['I']*(n-p-1))
    str2 = tuple(['Z']*p + ['Y'] + ['I']*(n-p-1))
    return {str1:0.5, str2:0.5j}
def a_p_dag(p,n):
    str1 = tuple(['Z']*p + ['X'] + ['I']*(n-p-1))
    str2 = tuple(['Z']*p + ['Y'] + ['I']*(n-p-1))
    return {str1:0.5, str2:-0.5j}

def mul_strs(str1, str2):
    phase = 1
    res = []
    for i in range(len(str1)):
        mul = mul_table[(str1[i], str2[i])]
        phase *= mul[0]
        res.append(mul[1])
    return (phase, tuple(res))

def mul_pauli(t1, t2):
    res = {}
    t1 = t1.items()
    t2 = t2.items()
    for (str1, c1) in t1:
        for (str2, c2) in t2:
            mul = mul_strs(str1,str2)
            coeff = mul[0]*c1*c2
            str_res = mul[1]
            res[str_res]=res.get(str_res,0.0) + coeff
    return res

def add_pauli(t1,t2):
    res = t2.copy()
    t1 = t1.items()
    for (str1, c1) in t1:
        if str1 in res:
            res[str1] += c1
            if res[str1] == 0:
                del res[str1]
        else:
            res[str1] = c1
    return res

#### Arranging (E_nuc, h_so, g_so) into pauli form

In [70]:
def JW_map(E_nuc, h_so, g_so, eps = 1e-12):
    n = h_so.shape[0]
    H = {tuple(['I']*n): E_nuc}       # start with the constant term

    # ------- one-body part  Σ h_pq a†_p a_q -------
    for p in range(n):
        for q in range(n):
            coeff = h_so[p, q]
            if coeff==0:continue
            term_pq = mul_pauli(a_p_dag(p, n), a_p(q, n))
            for s, c in term_pq.items():
                H[s] = H.get(s, 0.0) + coeff * c

    # ------- two-body part  ½ Σ g_pqrs a†_p a†_q a_s a_r -------
    for p in range(n):
        for q in range(n):
            for r in range(n):
                for s in range(n):
                    coeff = 0.5 * g_so[p, q, r, s]
                    if coeff==0:continue
                    term = mul_pauli(
                        mul_pauli(a_p_dag(p, n), a_p_dag(q, n)),
                        mul_pauli(a_p(s, n), a_p(r, n))
                    )
                    for st, ct in term.items():
                        H[st] = H.get(st, 0.0) + coeff * ct

    # prune numerically-zero entries
    return {k: v for k, v in H.items() if abs(v) > eps}

In [71]:
JW_map(E_nuc, h_so, g_so)

{('I', 'I', 'I', 'I'): np.complex128(0.07286973894478338+0j),
 ('Z', 'I', 'I', 'I'): np.complex128(0.20700459892734405+0j),
 ('I', 'Z', 'I', 'I'): np.complex128(0.20700459892734402+0j),
 ('I', 'I', 'Z', 'I'): np.complex128(0.008035886505754278+0j),
 ('I', 'I', 'I', 'Z'): np.complex128(0.008035886505754278+0j),
 ('Z', 'Z', 'I', 'I'): np.complex128(0.08446376935043953+0j),
 ('Y', 'X', 'Y', 'X'): np.complex128(0.030228158154441565+0j),
 ('Y', 'X', 'X', 'Y'): np.complex128(0.052844558127470485+0j),
 ('Y', 'Y', 'X', 'X'): np.complex128(-0.052844558127470485+0j),
 ('Y', 'Y', 'Y', 'Y'): np.complex128(0.030228158154441565+0j),
 ('X', 'X', 'X', 'X'): np.complex128(0.030228158154441565+0j),
 ('X', 'X', 'Y', 'Y'): np.complex128(-0.052844558127470485+0j),
 ('X', 'Y', 'Y', 'X'): np.complex128(0.052844558127470485+0j),
 ('X', 'Y', 'X', 'Y'): np.complex128(0.030228158154441565+0j),
 ('Z', 'I', 'I', 'Z'): np.complex128(0.022616399973028924+0j),
 ('I', 'Z', 'Z', 'I'): np.complex128(0.022616399973028924